In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# Load positive and negative datasets
positive_data = pd.read_csv("MIBiG.pfam.tsv", sep="\t")
negative_data = pd.read_csv("GeneSwap_Negatives.pfam.tsv", sep="\t")

# Add labels: 1 for positive (BGC), 0 for negative (non-BGC)
positive_data["label"] = 1
negative_data["label"] = 0

# Combine datasets
combined_data = pd.concat([positive_data, negative_data], ignore_index=True)

# Create vocabulary of Pfam IDs
vocab = set(combined_data["pfam_id"].unique())
vocab_size = 9633  # As provided
pfam_to_idx = {pfam: idx + 1 for idx, pfam in enumerate(vocab)}  # +1 to reserve 0 for padding
idx_to_pfam = {idx: pfam for pfam, idx in pfam_to_idx.items()}

# Group by sequence_id or contig_id to create sequences
def create_sequences(data, group_col, pfam_col, label_col):
    sequences = []
    labels = []
    for group, group_data in data.groupby(group_col):
        sequences.append([pfam_to_idx[pfam] for pfam in group_data[pfam_col]])
        labels.append(group_data[label_col].iloc[0])  # All rows in a group have the same label
    return sequences, labels

# Create sequences for positive and negative data
positive_sequences, positive_labels = create_sequences(positive_data, "sequence_id", "pfam_id", "label")
negative_sequences, negative_labels = create_sequences(negative_data, "contig_id", "pfam_id", "label")

# Combine sequences and labels
all_sequences = positive_sequences + negative_sequences
all_labels = positive_labels + negative_labels

# Pad sequences to the same length
max_seq_length = 373  # As provided
padded_sequences = [seq + [0] * (max_seq_length - len(seq)) for seq in all_sequences]  # 0 is padding index

# Convert to PyTorch tensors
X = torch.tensor(padded_sequences, dtype=torch.long)
y = torch.tensor(all_labels, dtype=torch.long)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create PyTorch Dataset
class PfamDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx]

train_dataset = PfamDataset(X_train, y_train)
val_dataset = PfamDataset(X_val, y_val)

# Create DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [2]:
class BiLSTM_Classifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(BiLSTM_Classifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim, padding_idx=0)  # +1 for padding
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)  # Fully connected layer for classification

    def forward(self, x):
        embeds = self.embedding(x)  # Shape: (batch_size, sequence_length, embedding_dim)
        lstm_out, _ = self.bilstm(embeds)  # Shape: (batch_size, sequence_length, hidden_dim)
        
        # Global average pooling over the sequence length
        pooled = lstm_out.mean(dim=1)  # Shape: (batch_size, hidden_dim)
        
        # Final classification layer
        logits = self.fc(pooled)  # Shape: (batch_size, num_classes)
        return logits

# Define model hyperparameters
vocab_size = 9633  # As provided
embedding_dim = 128
hidden_dim = 256
num_classes = 2  # Binary classification: BGC (1) or non-BGC (0)

# Initialize model, loss, and optimizer
model = BiLSTM_Classifier(vocab_size, embedding_dim, hidden_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [3]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(sequences)  # Shape: (batch_size, num_classes)
        
        # Calculate loss
        loss = criterion(outputs, labels)  # labels shape: (batch_size)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            outputs = model(sequences)  # Shape: (batch_size, num_classes)
            
            # Calculate validation loss
            val_loss += criterion(outputs, labels).item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")

Epoch 1/10, Loss: 0.24915999892127788
Validation Loss: 0.12423254240696367, Accuracy: 0.945934791580685
Epoch 2/10, Loss: 0.09412145120992323
Validation Loss: 0.08217714595191769, Accuracy: 0.976062732150227
Epoch 3/10, Loss: 0.05743977088840808
Validation Loss: 0.0791475266696101, Accuracy: 0.9715229054890632
Epoch 4/10, Loss: 0.037460970719999605
Validation Loss: 0.06545809579792579, Accuracy: 0.9773008666941808
Epoch 5/10, Loss: 0.030582942562285356
Validation Loss: 0.06785109426709823, Accuracy: 0.977713578208832
Epoch 6/10, Loss: 0.02289315675412287
Validation Loss: 0.07286800754158512, Accuracy: 0.9801898472967395
Epoch 7/10, Loss: 0.0165732242337969
Validation Loss: 0.0643896695715069, Accuracy: 0.9830788278992983
Epoch 8/10, Loss: 0.014251269141787266
Validation Loss: 0.0704914813127528, Accuracy: 0.9797771357820884
Epoch 9/10, Loss: 0.023953468759692884
Validation Loss: 0.08446902422675569, Accuracy: 0.9739991745769707
Epoch 10/10, Loss: 0.01843862373650031
Validation Loss: 0.

In [4]:
# Load test data
test_data = pd.read_csv("test.csv")
test_sequences = [[pfam_to_idx[pfam] for pfam in test_data["pfam_id"]]]
padded_test_sequences = [seq + [0] * (max_seq_length - len(seq)) for seq in test_sequences]
X_test = torch.tensor(padded_test_sequences, dtype=torch.long)

# Predict
model.eval()
with torch.no_grad():
    outputs = model(X_test)  # Shape: (batch_size, num_classes)
    _, predicted = torch.max(outputs, 1)  # Get predicted class indices

# Convert predictions to labels
predicted_labels = predicted.tolist()
predicted_bgc = [idx_to_pfam[idx] for idx in predicted_labels if idx != 0]  # Exclude padding

# Output predicted BGC clusters
print("Predicted BGC Clusters:", predicted_bgc)

Predicted BGC Clusters: ['PF14871']


# 2nd Trial

In [8]:
import torch
import torch.nn as nn
from torchcrf import CRF

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(BiLSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim, padding_idx=0)  # +1 for padding
        self.bilstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)  # Fully connected layer for tag scores
        self.crf = CRF(num_classes, batch_first=True)  # CRF layer

    def forward(self, x, labels=None):  # Add `labels` as an optional argument
        embeds = self.embedding(x)  # Shape: (batch_size, sequence_length, embedding_dim)
        lstm_out, _ = self.bilstm(embeds)  # Shape: (batch_size, sequence_length, hidden_dim)
        tag_scores = self.fc(lstm_out)  # Shape: (batch_size, sequence_length, num_classes)

        if labels is not None:
            # Training: Compute CRF loss
            loss = -self.crf(tag_scores, labels)  # Negative log likelihood
            return loss
        else:
            # Inference: Decode the best sequence of tags
            return self.crf.decode(tag_scores)

In [9]:
# Define model hyperparameters
vocab_size = 9633  # As provided
embedding_dim = 128
hidden_dim = 256
num_classes = 2  # Binary classification: BGC (1) or non-BGC (0)

# Initialize the BiLSTM_CRF model
model = BiLSTM_CRF(vocab_size, embedding_dim, hidden_dim, num_classes)

In [11]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass: Compute CRF loss
        loss = model(sequences, labels)  # Pass labels to compute CRF loss
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for sequences, labels in val_loader:
            # Forward pass: Compute CRF loss
            loss = model(sequences, labels)
            val_loss += loss.item()
            
            # Decode the best sequence of tags
            predicted_tags = model(sequences)  # Shape: (batch_size, sequence_length)
            
            # Flatten labels and predictions for accuracy calculation
            true_tags = labels.view(-1).tolist()
            pred_tags = [tag for seq in predicted_tags for tag in seq]
            
            # Calculate accuracy
            correct += sum(t1 == t2 for t1, t2 in zip(true_tags, pred_tags))
            total += len(true_tags)

    print(f"Validation Loss: {val_loss / len(val_loader)}, Accuracy: {correct / total}")

ValueError: the first two dimensions of emissions and tags must match, got (32, 373) and (32,)